Import modules

In [ ]:
import geopandas as gpd
import numpy as np
import imageio
import os
import itertools

from matplotlib import pyplot, cm, colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from multiprocess import Pool, cpu_count

import utils

Prepare environment and load data

In [ ]:
world_shape_df=gpd.read_file('maps/ne_10m_admin_0_sovereignty.shp')

world_states_shape_df = gpd.read_file('maps/ne_10m_admin_1_states_provinces.shp')
russia_shape_df = world_states_shape_df[world_states_shape_df.admin == 'Russia']

world_report_df = utils.storage.get_countries_report()
russia_report_df = utils.storage.get_regions_report('Russia')

if not os.path.exists('./assets'):
    os.mkdir('./assets')
    
if not os.path.exists('./temp'):
    os.mkdir('./temp')

Process data - add more metrics if needed

In [ ]:
world_shape_df = world_shape_df.to_crs({'init' :'epsg:4326'})
russia_shape_df = russia_shape_df.to_crs({'init': 'epsg:5940'})

world_stats = utils.storage.get_countries_stats()

def augment_report(df, country, region = None):
    name = country
    if region:
        name = region

    for column, per in itertools.product(['Confirmed', 'Deaths', 'Confirmed_Change', 'Deaths_Change'],[1,1000,100_000]):
        suffix = 'per_capita'
        if per == 1000:
            suffix = 'per_1k'
        elif per == 100_000:
            suffix = 'per_100k'
            
        df.loc[df.Name == name, column+'_'+suffix] = utils.data.per_value(
            df.loc[df.Name == name, column], 
            country,
            region,
            per=per)
    
    for column in ['Confirmed', 'Deaths', 'Confirmed_Change', 'Deaths_Change']:
        df.loc[df.Name == name, column+'_Norm'] = utils.data.normalize(df.loc[df.Name == name, column])

for country in utils.storage.get_countries():
    augment_report(world_report_df, country)
    world_report_df.loc[world_report_df.Name == country, 'Continent'] = world_stats.loc[country, 'Continent']
    
for region in utils.storage.get_country_regions('Russia'):
    augment_report(russia_report_df, 'Russia', region)

Rename some countries and regions or remove them to join two shape and data dataframes

In [ ]:
for data, shape in [
    ('North Macedonia', 'Macedonia'),
    ('Holy See', 'Vatican'),
    ('Cote d\'Ivoire', 'Ivory Coast'),
    ('Congo (Kinshasa)', 'Democratic Republic of the Congo'),
    ('Congo (Brazzaville)', 'Republic of the Congo'),
    ('Bahamas', 'The Bahamas'),
    ('Serbia', 'Republic of Serbia'),
    ('Sao Tome and Principe', 'São Tomé and Principe'),
    ('Tanzania', 'United Republic of Tanzania'),
    ('UK', 'United Kingdom'),
    ('US', 'United States of America')
]:
    world_report_df.loc[world_report_df.Name == data, 'Name'] = shape
    
for to_remove in ['West Bank and Gaza', 'Timor-Leste', 'Hong Kong', 'Macau']:
    world_report_df = world_report_df[world_report_df.Name != to_remove]
    
world_shape_df.loc[world_shape_df['ADMIN'] == 'Baykonur Cosmodrome', 'ADMIN'] = 'Kazakhstan'
    
    
    
russia_shape_df.loc[1442, 'name_ru'] = "Алтайский край"
russia_shape_df.dropna(subset = ['name_ru'], inplace = True)

for data, shape in [
    ('Крым', 'Автономная Республика Крым'),
    ('Алтай', 'Республика Алтай'),
    ('Еврейская АО', 'Еврейская автономная область'),
    ('Карачаево-Черкессия', 'Карачаево-Черкесия'),
    ('Карелия', 'Республика Карелия'),
    ('Коми', 'Республика Коми'),
    ('Ненецкий АО', 'Ненецкий автономный округ'),
    ('Северная Осетия', 'Республика Северная Осетия-Алания'),
    ('Саха (Якутия)', 'Якутия'),
    ('ХМАО – Югра', 'Ханты-Мансийский автономный округ — Югра'),
    ('Чукотский АО', 'Чукотский автономный округ'),
    ('Ямало-Ненецкий АО', 'Ямало-Ненецкий автономный округ'),
]:
    russia_shape_df.loc[russia_shape_df.name_ru == shape, 'name_ru'] = data

russia_shape_df['name_ru'] = russia_shape_df['name_ru'].apply(lambda x: x[:-4] +'.' if x.endswith('область') else x)

Preparations for images rendering:
- Set start, fin, step variables
- Set pool_size. CAUTION: Big values could lead to out of memory exceptions and to kernel crash
- Set cmap

In [ ]:
start = utils.first_day
rus_start = utils.str_to_datetime('15-03-2020')
step = utils.one_day
fin =  utils.last_day
pool_size = max(1, int(cpu_count()*.75))
sea_color = '#CBE8FE'

cmap = colors.LinearSegmentedColormap.from_list(
    'test', [(0, 'darkGreen'), (0.2, 'forestgreen'),
             (0.4, 'yellowgreen'), (0.5, 'gold'), (0.7, 'orange'),
             (0.9, 'orangered'), (1, 'crimson')])

def get_fig_axes():
    fig = pyplot.figure(figsize=(24,14))
    fig.set_tight_layout({"pad":0.1})
        
    ax = fig.add_subplot(1,1,1)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="1%", pad=0.05)
    fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax,cax=cax,orientation='horizontal')
   
    return fig, ax

def draw_top_10(df, name_column, data_column, ax, 
                name_name='Country', data_name = 'Cases',
                as_int = True, x0 = 0.01, y0 = 0.01):
    formatter = (lambda x: '{:d}'.format(int(x))) if as_int else (lambda x: '{:.2f}'.format(float(x)))
    table_data =list(
        df.dropna(subset=[data_column]).sort_values(
            by = data_column,
            ascending = False).head(10)[[name_column, data_column]].apply(
            lambda x: [x[name_column], formatter(x[data_column])], axis=1).values)
    
    count = len(table_data)
    
    if count > 0:
        table = ax.table(table_data,
                         colLabels=[name_name, data_name],
                         rowLabels=list(range(1, count+1)),
                         colWidths=[2, 1],
                         cellColours=np.reshape(np.repeat(sea_color, 2*count), (count, 2)),
                         rowColours = np.repeat(sea_color, count),
                         colColours = np.repeat(sea_color, 2),
                         bbox=[x0, y0, .2, .027*(count+1)])
        
        table.auto_set_font_size(False)
        table.set_fontsize(10)

Render frames asynchronously

In [ ]:
column_name = 'Confirmed_per_100k'
vmin = 0
vmax = world_report_df[column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'world_confirmed_100k'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_xlim(-180,180)
    ax.set_ylim(-90,90)
    ax.set_axis_off()
    
    day = start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = world_shape_df.set_index('ADMIN').join(world_report_df[world_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'ADMIN', column_name, ax, data_name='Cases per 100k', as_int=False, y0=0.1)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
        
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Confirmed'
vmin = 1
vmax = world_report_df[column_name].quantile(.97)
norm = colors.LogNorm(vmin,vmax)
folder_name = 'world_confirmed'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_xlim(-180,180)
    ax.set_ylim(-90,90)
    ax.set_axis_off()
    
    day = start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = world_shape_df.set_index('ADMIN').join(world_report_df[world_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'ADMIN', column_name, ax, y0=0.1)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
        
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Deaths_per_100k'
vmin = 0
vmax = world_report_df[column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'world_deaths_100k'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_xlim(-180,180)
    ax.set_ylim(-90,90)
    ax.set_axis_off()
    
    day = start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = world_shape_df.set_index('ADMIN').join(world_report_df[world_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,250), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'ADMIN', column_name, ax, data_name='Cases per 100k', as_int=False, y0=0.1)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
        
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Deaths'
vmin = 0
vmax = world_report_df[column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'world_deaths'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_xlim(-180,180)
    ax.set_ylim(-90,90)
    ax.set_axis_off()
    
    day = start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = world_shape_df.set_index('ADMIN').join(world_report_df[world_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'ADMIN', column_name, ax, y0=0.1)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
        
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Deaths_per_100k'
vmin = 0
vmax = world_report_df.loc[world_report_df.Continent=='Europe', column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'europe_deaths_100k'
europe_shape_df = world_shape_df.loc[world_shape_df.ADMIN.isin(set(world_report_df.loc[world_report_df.Continent=='Europe','Name']))]

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_xlim(-12,40)
    ax.set_ylim(30,73)
    ax.set_axis_off()
    
    day = start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = europe_shape_df.set_index('ADMIN').join(world_report_df[world_report_df.Date == day].set_index('Name')[[column_name, 'Continent']]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='figure pixels', fontsize=24)
    draw_top_10(temp_df, 'ADMIN', column_name, ax, data_name='Cases per 100k', as_int=False, x0=-0.215)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
    
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Deaths'
vmin = 0
vmax = world_report_df.loc[world_report_df.Continent=='Europe', column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'europe_deaths'
europe_shape_df = world_shape_df.loc[world_shape_df.ADMIN.isin(set(world_report_df.loc[world_report_df.Continent=='Europe','Name']))]

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_xlim(-12,40)
    ax.set_ylim(30,73)
    ax.set_axis_off()
    
    day = start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = europe_shape_df.set_index('ADMIN').join(world_report_df[world_report_df.Date == day].set_index('Name')[[column_name, 'Continent']]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='figure pixels', fontsize=24)
    draw_top_10(temp_df, 'ADMIN', column_name, ax, x0=-0.215)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
    
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Confirmed_per_100k'
vmin = 0
vmax = world_report_df.loc[world_report_df.Continent=='Europe', column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'europe_confirmed_100k'
europe_shape_df = world_shape_df.loc[world_shape_df.ADMIN.isin(set(world_report_df.loc[world_report_df.Continent=='Europe','Name']))]

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_xlim(-12,40)
    ax.set_ylim(30,73)
    ax.set_axis_off()
    
    day = start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = europe_shape_df.set_index('ADMIN').join(world_report_df[world_report_df.Date == day].set_index('Name')[[column_name, 'Continent']]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=0.5)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='figure pixels', fontsize=24)
    draw_top_10(temp_df, 'ADMIN', column_name, ax, data_name='Cases per 100k', as_int=False, x0=-0.215)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
    
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Confirmed'
vmin = 0
vmax = world_report_df.loc[world_report_df.Continent=='Europe', column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'europe_confirmed'
europe_shape_df = world_shape_df.loc[world_shape_df.ADMIN.isin(set(world_report_df.loc[world_report_df.Continent=='Europe','Name']))]

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_xlim(-12,40)
    ax.set_ylim(30,73)
    ax.set_axis_off()
    
    day = start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = europe_shape_df.set_index('ADMIN').join(world_report_df[world_report_df.Date == day].set_index('Name')[[column_name, 'Continent']]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=0.5)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='figure pixels', fontsize=24)
    draw_top_10(temp_df, 'ADMIN', column_name, ax, x0=-0.215)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
        
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Confirmed'
vmin = 0
vmax = russia_report_df[column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'russia_confirmed'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_axis_off()
    
    day = rus_start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = russia_shape_df.set_index('name_ru').join(russia_report_df[russia_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'name_ru', column_name, ax, name_name='Regions')
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
    
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-rus_start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Confirmed_per_100k'
vmin = 0
vmax = russia_report_df[column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'russia_confirmed_100k'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_axis_off()
    
    day = rus_start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = russia_shape_df.set_index('name_ru').join(russia_report_df[russia_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'name_ru', column_name, ax, name_name='Regions', data_name='Cases per 100k', as_int=False)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
    
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-rus_start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Confirmed_Change'
vmin = 0
vmax = russia_report_df[column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'russia_confirmed_change'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_axis_off()
    
    day = rus_start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = russia_shape_df.set_index('name_ru').join(russia_report_df[russia_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'name_ru', column_name, ax, name_name='Regions')
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
    
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-rus_start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Confirmed_Change_per_100k'
vmin = 0
vmax = russia_report_df[column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'russia_confirmed_change_100k'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_axis_off()
    
    day = rus_start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = russia_shape_df.set_index('name_ru').join(russia_report_df[russia_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'name_ru', column_name, ax, name_name='Regions', data_name='Cases per 100k', as_int=False)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
    
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-rus_start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Deaths'
vmin = 0
vmax = russia_report_df[column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'russia_deaths'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_axis_off()
    
    day = rus_start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = russia_shape_df.set_index('name_ru').join(russia_report_df[russia_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'name_ru', column_name, ax, name_name='Regions')
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
    
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-rus_start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Deaths_per_100k'
vmin = 0
vmax = russia_report_df[column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'russia_deaths_100k'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_axis_off()
    
    day = rus_start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = russia_shape_df.set_index('name_ru').join(russia_report_df[russia_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'name_ru', column_name, ax, name_name='Regions', data_name='Cases per 100k', as_int=False)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
    
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-rus_start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Deaths_Change'
vmin = 0
vmax = russia_report_df[column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'russia_deaths_change'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_axis_off()
    
    day = rus_start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = russia_shape_df.set_index('name_ru').join(russia_report_df[russia_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'name_ru', column_name, ax, name_name='Regions')
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')    

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')
    
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-rus_start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

In [ ]:
column_name = 'Deaths_Change_per_100k'
vmin = 0
vmax = russia_report_df[column_name].quantile(.95)
norm = colors.Normalize(vmin,vmax)
folder_name = 'russia_deaths_change_100k'

fig, ax = get_fig_axes()

def process_day(i):
    ax.set_axis_off()
    
    day = rus_start + step * i
    annotation = day.date().strftime('%Y-%m-%d')
    
    temp_df = russia_shape_df.set_index('name_ru').join(russia_report_df[russia_report_df.Date == day].set_index('Name')[column_name]).reset_index()
    
    temp_df.plot(ax=ax, color='white', edgecolor='black', linewidth=1)
    temp_df.plot(ax=ax, column=column_name, linewidth=0, vmin=vmin, vmax=vmax, cmap=cmap, norm=norm, alpha=0.9)
    
    ax.annotate(annotation, xy=(20,850), xycoords='axes pixels', fontsize=24)
    draw_top_10(temp_df, 'name_ru', column_name, ax, name_name='Regions', data_name='Cases per 100k', as_int=False)
        
    fig.savefig(f'./temp/{folder_name}/{day.date().strftime("%Y-%m-%d")}.jpg', dpi=72, facecolor='#CBE8FE')  

    ax.clear()

if __name__ == '__main__':
    if not os.path.exists(f'./temp/{folder_name}'):
        os.mkdir(f'./temp/{folder_name}')

    
    with Pool(pool_size) as p:
        results = list()
        
        for i in range((fin-rus_start).days):
            results.append(p.apply_async(process_day ,[i]))
            
        for r in tqdm(results): 
            r.wait()
            
    pyplot.close('all')

Make video from frames

In [ ]:
def make_video(name):
    images = list(os.listdir(f'./temp/{name}'))

    with imageio.get_writer(f'./assets/{name}.mp4', mode='I', fps=6) as writer:
        for i in range(len(images)):
            image = imageio.imread(os.path.join(os.path.abspath(f'./temp/{name}'),images[i]))
            writer.append_data(image)
            

if __name__ == '__main__':
    with Pool(int(pool_size*.5)) as p:
        results = list()
        
        for name in os.listdir('./temp'):
            results.append(p.apply_async(make_video,[name]))
            
        for r in tqdm(results): 
            r.wait()
